In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, add_accuracy_to_epochs
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import json


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MOVE ALL FUNCTIONS TO THE TOP!

In [2]:
def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

#delete this once we know the import of this function works
def add_accuracy_to_epochs(epochs, accuracy_array):
    """
    Adds accuracy data from accuracy_array to the metadata of epochs.
    Assumes the order of trials in accuracy_array matches the order in epochs.
    """
    if epochs.metadata is None:
        # Create a new DataFrame if no metadata exists
        epochs.metadata = pd.DataFrame(index=range(len(epochs)))
    
    # Ensure the accuracy_array length matches the number of epochs
    print('length of accuracy array:', len(accuracy_array))
    print('length of epochs:', len(epochs))
    assert len(accuracy_array) == len(epochs), "Mismatch in number of trials and accuracy data length."
    
    # Add the accuracy array as a new column in the metadata
    epochs.metadata['accuracy'] = accuracy_array

    # Reset the index to ensure it's sequential starting from 0
    epochs.metadata.reset_index(drop=True, inplace=True)
    
    return epochs

def create_subjects_mne_objects_dict(subjects, output_names_conditions, task, combined_data, acc_array, LAB_root=None):
    """
    Adjusted to handle multiple conditions per output name, with multiple condition columns.

    Parameters:
    - subjects: List of subject IDs.
    - output_names_conditions: Dictionary where keys are output names and values are dictionaries
        of condition column names and their required values.
    - task: Task identifier.
    - combined_data: DataFrame with combined behavioral and trial information.
    - acc_array: dict of numpy arrays of 0 for incorrect and 1 for correct trials for each subject
    - LAB_root: Root directory for data (optional).
    """
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")
        sub_mne_objects = {}
        for output_name, conditions in output_names_conditions.items():
            print(f"  Loading output: {output_name} with conditions: {conditions}")
            
            # Build the filtering condition
            sub_without_zeroes = "D" + sub[1:].lstrip('0') 
            condition_filter = (combined_data['subject_ID'] == sub) # this previously indexed using sub_without_zeroes, but now just uses sub. 3/17.
                    
            for condition_column, condition_value in conditions.items():
                if isinstance(condition_value, list):
                    # If the condition needs to match any value in a list
                    condition_filter &= combined_data[condition_column].isin(condition_value)
                else:
                    # If the condition is a single value
                    condition_filter &= (combined_data[condition_column] == condition_value)
            
            # Filter combinedData for the specific subject and conditions
            subject_condition_data = combined_data[condition_filter]
            
            # Load MNE objects and update with accuracy data
            mne_objects = load_mne_objects(sub, output_name, task, LAB_root)
            
            if sub in acc_array:
                trial_counts = subject_condition_data['trialCount'].values.astype(int)
                accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
                # Now pass trial_counts along with accuracy_data
                mne_objects['HG_ev1_rescaled'] = add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

In [3]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [ ]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0071', 'D0077', 'D0090', 'D0100', 'D0102', 'D0103']
# subjects = ['D0103'] #testing cuz d0065 being weird

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# # Save to a JSON file. Uncomment when actually running.
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

### load subjects electrodes to rois dict

In [4]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [5]:
# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_c25_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.

In [ ]:
HG_ev1_rescaled_D0057_c.get_data().shape

load accuracy arrays so we can filter by only accurate trials  
combine this code into add_accuracy_to_epochs later!

In [6]:
# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        subject_id = file.split('_')[0]  # Extract subject ID from the file name
        if subject_id != 'D0107':  # Check if the subject ID is not D0107. Skip D0107 for now because it's not preprocessed yet 3/17.
            # Construct the full file path
            file_path = os.path.join(npy_directory, file)
            # Load the numpy array from the file
            acc_array[subject_id] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.

In [7]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

# Skip D0107 for now cuz it's not preprocessed yet 3/17.
combined_data = combined_data[combined_data['subject_ID'] != 'D0107']

In [8]:
# Define a function to map blockType to congruencyProportion and switchProportion
def map_block_type(row):
    if row['blockType'] == 'A':
        return pd.Series(['25%', '25%'])
    elif row['blockType'] == 'B':
        return pd.Series(['25%', '75%'])
    elif row['blockType'] == 'C':
        return pd.Series(['75%', '25%'])
    elif row['blockType'] == 'D':
        return pd.Series(['75%', '75%'])
    else:
        return pd.Series([None, None])

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(map_block_type, axis=1)

### load evoked and stuff for all subjects in a dictionary

In [9]:
# # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # make sure to use the correct column names and values that match with what combinedData uses.
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": ["s1", "s2"]  # Example where switchType needs to match any value in the list
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# congruency
# output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#     }
# }

# switch
# output_names = ["Stimulus_r25and75_fixationCrossBase_1sec_mirror", "Stimulus_s25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_r25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#     },
#     "Stimulus_s25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#     }
# }

# #  ir vs is
# output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  cr vs cs
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# #  is vs cs
# output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "s"
#     },
#     "Stimulus_is_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

# #  ir vs cr
# output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "r"
#     }
# }

# # all interaction effects (run this with the anova code. Ugh make everything more modular later.)
output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror", "Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]

output_names_conditions = {
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "r"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "s"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "r"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "s"
    }
}

# # block interaction contrasts for lwpc
# output_names = ["Stimulus_c25_fixationCrossBase_1sec_mirror", "Stimulus_c75_fixationCrossBase_1sec_mirror",  \
#                 "Stimulus_i25_fixationCrossBase_1sec_mirror", "Stimulus_i75_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_c25_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_c75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "congruencyProportion": "25%"
#     },
#     "Stimulus_i25_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "congruencyProportion": "75%"
#     },
#     "Stimulus_i75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "congruencyProportion": "25%"
#     },
# }

# # block interaction contrasts for lwps
# output_names = ["Stimulus_s25_fixationCrossBase_1sec_mirror", "Stimulus_s75_fixationCrossBase_1sec_mirror",  \
#                 "Stimulus_r25_fixationCrossBase_1sec_mirror", "Stimulus_r75_fixationCrossBase_1sec_mirror"]

# output_names_conditions = {
#     "Stimulus_s25_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_s75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_r25_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_r75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
# }

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array)

Loading data for subject: D0057
  Loading output: Stimulus_ir_fixationCrossBase_1sec_mirror with conditions: {'congruency': 'i', 'switchType': 'r'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_ir_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
103 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_ir_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\fre

FileNotFoundError: File does not exist: "C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0069\D0069_Stimulus_ir_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif"

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [10]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


### do time perm cluster stats

In [ ]:
# Initialize a dictionary to hold mappings
overall_electrode_mapping = []

# Initialize lists for storing data
output_0_data_trialAvg_list = []
output_1_data_trialAvg_list = []
output_0_data_timeAvg_firstHalfSecond_list = []
output_1_data_timeAvg_firstHalfSecond_list = []
output_0_data_timeAvg_secondHalfSecond_list = []
output_1_data_timeAvg_secondHalfSecond_list = []
output_0_data_timeAvg_fullSecond_list = []
output_1_data_timeAvg_fullSecond_list = []

# Time windows
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096


for sub in subjects:
    sig_electrodes = sig_chans_per_subject.get(sub, [])

    for electrode in sig_electrodes:
        # For each significant electrode, append a tuple to the mapping list
        # Tuple format: (Subject ID, Electrode Name, Index in List)
        # The index can be the current length of the list before appending
        index = len(overall_electrode_mapping)
        overall_electrode_mapping.append((sub, electrode, index))  
        
    # Load trial-level data for the current condition and pick significant electrodes
    output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
    output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)

    # Calculate averages for each time window
    trial_avg_0, trial_std_0, time_avg_0_firstHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
    trial_avg_1, trial_std_1, time_avg_1_firstHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
    _, _, time_avg_0_secondHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
    _, _, time_avg_1_secondHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
    _, _, time_avg_0_fullSecond = filter_and_average_epochs(output_0_epochs, start_idx_fullSecond, end_idx_fullSecond)
    _, _, time_avg_1_fullSecond = filter_and_average_epochs(output_1_epochs, start_idx_fullSecond, end_idx_fullSecond)

    # Append the results to their respective lists
    output_0_data_trialAvg_list.append(trial_avg_0)
    output_1_data_trialAvg_list.append(trial_avg_1)
    output_0_data_timeAvg_firstHalfSecond_list.append(time_avg_0_firstHalfSecond)
    output_1_data_timeAvg_firstHalfSecond_list.append(time_avg_1_firstHalfSecond)
    output_0_data_timeAvg_secondHalfSecond_list.append(time_avg_0_secondHalfSecond)
    output_1_data_timeAvg_secondHalfSecond_list.append(time_avg_1_secondHalfSecond)
    output_0_data_timeAvg_fullSecond_list.append(time_avg_0_fullSecond)
    output_1_data_timeAvg_fullSecond_list.append(time_avg_1_fullSecond)

# After collecting all data, concatenate across subjects for each condition
concatenated_trialAvg_data = {}
concatenated_timeAvg_firstHalfSecond_data = {}
concatenated_timeAvg_secondHalfSecond_data = {}
concatenated_timeAvg_fullSecond_data = {}


concatenated_trialAvg_data = {
    'output_0': np.concatenate(output_0_data_trialAvg_list, axis=0),
    'output_1': np.concatenate(output_1_data_trialAvg_list, axis=0)
}

# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}

for output in ['output_0', 'output_1']:
    trialAvg_data = concatenated_trialAvg_data[output]
    overall_averages[output] = np.nanmean(trialAvg_data, axis=0)
    overall_sems[output] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])

time_perm_cluster_results = time_perm_cluster(
    concatenated_trialAvg_data['output_0'],
    concatenated_trialAvg_data['output_1'], 0.05, n_jobs=6
)

### do window stats  
use the time avg outputs from previous cell  
use fdr correction after comparing output 0 and output 1 for each electrode to get a p-values list  

DO A SHUFFLE INSTEAD OF PAIRED T-TEST AS OF 2/7/24


shuffle test (perm test). This basically time perm cluster but avg across time.

do perm testing

In [ ]:
# Assuming the functions perform_permutation_test_within_electrodes and perform_permutation_test_across_electrodes return lists of p-values (and get loaded in properly)
p_values = {}
# rois = ['dlpfc', 'acc', 'parietal']
# for roi in rois:
# Initialize p_values[roi] as a dictionary. Initialize dicts for all time windows.
p_values = {}
p_values['firstHalfSecond'] = {}
p_values['secondHalfSecond'] = {}
p_values['fullSecond'] = {}

# Perform the tests and store results
p_values['firstHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_firstHalfSecond_list, output_1_data_timeAvg_firstHalfSecond_list, n_permutations=10000)
p_values['firstHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_firstHalfSecond_list, output_1_data_timeAvg_firstHalfSecond_list, n_permutations=10000)

p_values['secondHalfSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_secondHalfSecond_list, output_1_data_timeAvg_secondHalfSecond_list, n_permutations=10000)
p_values['secondHalfSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_secondHalfSecond_list, output_1_data_timeAvg_secondHalfSecond_list, n_permutations=10000)

p_values['fullSecond']['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_fullSecond_list, output_1_data_timeAvg_fullSecond_list, n_permutations=10000)
p_values['fullSecond']['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_fullSecond_list, output_1_data_timeAvg_fullSecond_list, n_permutations=10000)

all_p_values = {}
all_p_values['firstHalfSecond'] = []
all_p_values['secondHalfSecond'] = []
all_p_values['fullSecond'] = []

# the very last p-value is the across electrodes p-value, all other p-values are within electrode
for test_type in p_values['firstHalfSecond']:
    p = p_values['firstHalfSecond'][test_type]
    if isinstance(p, list):
        all_p_values['firstHalfSecond'].extend(p)
    else:  # Assume it's a single float value
        all_p_values['firstHalfSecond'].append(p)

for test_type in p_values['secondHalfSecond']:
    p = p_values['secondHalfSecond'][test_type]
    if isinstance(p, list):
        all_p_values['secondHalfSecond'].extend(p)
    else:  # Assume it's a single float value
        all_p_values['secondHalfSecond'].append(p)

for test_type in p_values[roi]['fullSecond']:
    p = p_values['fullSecond'][test_type]
    if isinstance(p, list):
        all_p_values['fullSecond'].extend(p)
    else:  # Assume it's a single float value
        all_p_values['fullSecond'].append(p)

# Apply FDR correction
_, adjusted_p_values_firstHalfSecond = multipletests(all_p_values['firstHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
_, adjusted_p_values_secondHalfSecond = multipletestsload(all_p_values['secondHalfSecond'], alpha=0.05, method='fdr_bh')[:2]
_, adjusted_p_values_fullSecond = multipletests(all_p_values['fullSecond'], alpha=0.05, method='fdr_bh')[:2]

# Incorporating adjusted p-values back into the structure is a bit more complex and depends on how you want to use them next

In [ ]:
p_values['parietal']

In [ ]:
p_values['acc']

In [ ]:
p_values['parietal']

integrate adjusted p values back in the p values dict  
oh i think the index map fixes the fact that the across just gets added to the end of the within p values in the all p values list. But that's still confusing. 3/11

In [ ]:
# Step 1: Build an index map while aggregating p-values
index_map = {'firstHalfSecond': [], 'secondHalfSecond': [], 'fullSecond': []}

# Step 1: Adjusted - Ensure all p-values are treated as lists
for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
    for test_type in ['within', 'across']:
        p_value_list = p_values[time_window][test_type]
        # Ensure p_value_list is actually a list
        if not isinstance(p_value_list, list):
            p_value_list = [p_value_list]
        for p_value in p_value_list:
            all_p_values[time_window].append(p_value)
            index_map[time_window].append((test_type))


# Step 3: Reintegrate adjusted p-values back into the p_values structure
# Using firstHalfSecond as an example
# Adjusted reintegration example for firstHalfSecond
for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
    adjusted_ps = locals()[f"adjusted_p_values_{time_window}"]  # Retrieve adjusted p-values using dynamic variable names
    for i, adjusted_p in enumerate(adjusted_ps):
        test_type = index_map[time_window][i]
        # Ensure the adjusted key and test_type key exist
        if 'adjusted' not in p_values[time_window]:
            p_values[time_window]['adjusted'] = {}
        if test_type not in p_values[time_window]['adjusted']:
            p_values[time_window]['adjusted'][test_type] = []
        p_values[time_window]['adjusted'][test_type].append(adjusted_p)

In [ ]:
p_values['acc']

hmm figure out if need to run this always and how its different than the other one..

### do 2x2 anova for interaction effects 
this requires reloading in all four conditions (four this time cuz interaction contrasts).  
ONLY RUN THIS WHEN LOADING IN THE FOUR INTERACTION CONTRASTS RIGHT NOW.  
Integrate with other stats and plotting and stuff later.

In [11]:
# Initialize a list to hold mappings for all electrodes across all subjects
overall_electrode_mapping = []

# Initialize lists for storing data for each output condition and time window
output_data_trialAvg_lists = {output_name: [] for output_name in output_names}
output_data_trialStd_lists = {output_name: [] for output_name in output_names}
output_data_timeAvg_firstHalfSecond_lists = {output_name: [] for output_name in output_names}
output_data_timeAvg_secondHalfSecond_lists = {output_name: [] for output_name in output_names}
output_data_timeAvg_fullSecond_lists = {output_name: [] for output_name in output_names}

# Time windows
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096

# Iterate over each subject
for sub in subjects:
    sig_electrodes = sig_chans_per_subject.get(sub, [])  # Get significant electrodes for the current subject

    # Map significant electrodes
    for electrode in sig_electrodes:
        # Append a tuple to the mapping list (Subject ID, Electrode Name, Index in List)
        index = len(overall_electrode_mapping)
        overall_electrode_mapping.append((sub, electrode, index))

    # Process data for each output condition
    for output_idx, output_name in enumerate(output_names):
        # Load trial-level data for the current condition and pick significant electrodes
        epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)

        # Calculate averages for each time window
        trial_avg, trial_std, time_avg_firstHalfSecond = filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
        _, _, time_avg_secondHalfSecond = filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
        _, _, time_avg_fullSecond = filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)

        # Store the results
        output_data_trialAvg_lists[output_name].append(trial_avg)
        output_data_trialStd_lists[output_name].append(trial_std)
        output_data_timeAvg_firstHalfSecond_lists[output_name].append(time_avg_firstHalfSecond)
        output_data_timeAvg_secondHalfSecond_lists[output_name].append(time_avg_secondHalfSecond)
        output_data_timeAvg_fullSecond_lists[output_name].append(time_avg_fullSecond)

# After collecting all data, concatenate across subjects for each roi and condition
concatenated_trialAvg_data = {}
concatenated_trialStd_data = {}

for output_name in output_names:
    concatenated_trialAvg_data[output_name] = np.concatenate(output_data_trialAvg_lists[output_name], axis=0)
    concatenated_trialStd_data[output_name] = np.concatenate(output_data_trialStd_lists[output_name], axis=0)

# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}

for output_name in output_names:
    trialAvg_data = concatenated_trialAvg_data[output_name]
    overall_averages[output_name] = np.nanmean(trialAvg_data, axis=0)
    overall_sems[output_name] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\misc_functions.py:113: RuntimeWarning: Mean of empty slice
  time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

In [12]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Example structure for organizing data for ANOVA with four conditions
data_for_anova = []

# Function to process and append data for ANOVA from time-averaged lists
# Adapted function to include Congruency and SwitchType
def process_and_append_data_for_anova_whole_brain_analysis(time_averaged_lists, time_window):
    for output_name in output_names:
        # Dynamically get condition types and their values for the current output_name
        conditions = output_names_conditions[output_name]
        
        for subject_index, subject_data in enumerate(time_averaged_lists[output_name]):
            subject_id = subjects[subject_index]
            print(subject_id)

            # Skip this subject if there are no significant electrodes for them in this ROI
            if subject_id not in sig_chans_per_subject or not sig_chans_per_subject[subject_id]:
                continue

            # Calculate the mean across trials for each electrode
            mean_activity_per_electrode = np.nanmean(subject_data, axis=0)
            # untested making this more modular 2/27
            for electrode_index, mean_activity in enumerate(mean_activity_per_electrode):
                electrode_name = sig_chans_per_subject[subject_id][electrode_index]

                # Prepare data dictionary, starting with fixed attributes
                data_dict = {
                    'SubjectID': subject_id,
                    'Electrode': electrode_name,
                    'TimeWindow': time_window,
                    'MeanActivity': mean_activity
                }

                # Dynamically add condition types and their values
                data_dict.update(conditions)

                # Append the organized data to the list
                data_for_anova.append(data_dict)

# Invoke the function for each time-averaged list
process_and_append_data_for_anova_whole_brain_analysis(output_data_timeAvg_firstHalfSecond_lists, "FirstHalfSecond")
process_and_append_data_for_anova_whole_brain_analysis(output_data_timeAvg_secondHalfSecond_lists, "SecondHalfSecond")
process_and_append_data_for_anova_whole_brain_analysis(output_data_timeAvg_fullSecond_lists, "FullSecond")

# Convert to DataFrame
df_for_anova = pd.DataFrame(data_for_anova)

D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103
D0057
D0059
D0063
D0065
D0069
D0071
D0077
D0090
D0094
D0100
D0102
D0103


now actually run anova

In [13]:
def convert_dataframe_to_serializable_format(df):
    """
    Convert a pandas DataFrame to a serializable format that can be used with json.dump.
    """
    return df.to_dict(orient='records')

# move this to misc functions 3/20!
def perform_modular_anova(df, time_window, save_dir, save_name):
    # Filter for a specific time window (I should probably make this not have a time_window input and just loop over all time windows like the within electrode code does)
    df_filtered = df[df['TimeWindow'] == time_window]

    # Dynamically construct the model formula based on condition keys
    condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
    formula_terms = ' + '.join([f'C({key})' for key in condition_keys])
    interaction_terms = ' * '.join([f'C({key})' for key in condition_keys])
    formula = f'MeanActivity ~ {formula_terms} + {interaction_terms}'

    # Define the model
    model = ols(formula, data=df_filtered).fit()

    # Perform the ANOVA
    anova_results = anova_lm(model, typ=2)

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, save_name)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(anova_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"ANOVA results saved to: {results_file_path}")

    # Print the results
    print(anova_results)

    return anova_results

congruency as function of congruency proportion

In [14]:
# Assuming output_names is a list of strings that includes the names of the outputs you're working with
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FirstHalfSecond", save_dir, 'firstHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")


ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\firstHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt
                                           sum_sq      df         F    PR(>F)
C(congruency)                            0.453277     1.0  8.976509  0.002755
C(congruencyProportion)                  0.229585     1.0  4.546612  0.033056
C(congruency):C(congruencyProportion)    0.087015     1.0  1.723201  0.189371
Residual                               169.262335  3352.0       NaN       NaN


In [15]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "SecondHalfSecond", save_dir, 'secondHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\secondHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt
                                           sum_sq      df         F    PR(>F)
C(congruency)                            0.015082     1.0  0.261866  0.608875
C(congruencyProportion)                  0.260262     1.0  4.518999  0.033593
C(congruency):C(congruencyProportion)    0.000724     1.0  0.012579  0.910706
Residual                               193.051523  3352.0       NaN       NaN


In [16]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FullSecond", save_dir, 'fullSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

ANOVA results saved to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\fullSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt
                                           sum_sq      df         F    PR(>F)
C(congruency)                            0.158430     1.0  3.639486  0.056510
C(congruencyProportion)                  0.244684     1.0  5.620910  0.017804
C(congruency):C(congruencyProportion)    0.017965     1.0  0.412693  0.520650
Residual                               145.915708  3352.0       NaN       NaN


switch type and switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FirstHalfSecond", save_dir, 'firstHalfSecond_switchType_switchProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")

switch type as function of switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "SecondHalfSecond", save_dir, 'secondHalfSecond_switchType_switchProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FullSecond", save_dir, 'fullSecond_switchType_switchProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")

congruency as function of switch type

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FirstHalfSecond", save_dir, 'firstHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_cr_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "SecondHalfSecond", save_dir, 'secondHalfSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_cr_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    perform_modular_anova(df_for_anova, "FullSecond", save_dir, 'fullSecond_congruency_congruencyProportion_ANOVAacrossElectrodes_wholeBrainAnalysis.txt')
else:
    print("The required output name 'Stimulus_cr_fixationCrossBase_1sec_mirror' is not in output_names.")

okay now do within-electrode anova too

if the new modular code works then apply it here too (change data_for_anova basically)

In [17]:
data_for_anova = []

def process_and_append_trial_data_for_anova_whole_brain_analysis(time_averaged_lists, time_window, output_names_conditions):
    for output_name, conditions in output_names_conditions.items():
        for subject_index, subject_data in enumerate(time_averaged_lists[output_name]):
            subject_id = subjects[subject_index]
            
            if subject_id not in sig_chans_per_subject or not sig_chans_per_subject[subject_id]:
                continue

            for trial_index, trial_data in enumerate(subject_data):
                if np.any(np.isnan(trial_data)) or len(trial_data) != len(sig_chans_per_subject[subject_id]):
                    continue

                for electrode_index, electrode_name in enumerate(sig_chans_per_subject[subject_id]):
                    activity = trial_data[electrode_index] if electrode_index < len(trial_data) else np.nan
                    
                    # Prepare the data dictionary
                    data_dict = {
                        'SubjectID': subject_id,
                        'Electrode': electrode_name,
                        'TimeWindow': time_window,
                        'Trial': trial_index + 1,
                        'Activity': activity
                    }
                    
                    # Dynamically add condition types and their values
                    data_dict.update(conditions)

                    data_for_anova.append(data_dict)

# Invoke the function for each time-averaged list
process_and_append_trial_data_for_anova_whole_brain_analysis(output_data_timeAvg_firstHalfSecond_lists, "FirstHalfSecond", output_names_conditions)
process_and_append_trial_data_for_anova_whole_brain_analysis(output_data_timeAvg_secondHalfSecond_lists, "SecondHalfSecond", output_names_conditions)
process_and_append_trial_data_for_anova_whole_brain_analysis(output_data_timeAvg_fullSecond_lists, "FullSecond", output_names_conditions)

# Convert to DataFrame
df_for_trial_level_anova = pd.DataFrame(data_for_anova)

In [ ]:
df_for_trial_level_anova

In [18]:
def extract_significant_effects(anova_table):
    """
    Extract significant effects and their p-values from the ANOVA results table,
    removing 'C(...)' from effect names and formatting them neatly.
    """
    significant_effects = []
    for effect in anova_table.index:
        p_value = anova_table.loc[effect, 'PR(>F)']
        if p_value < 0.05:
            # Remove 'C(' and ')' from the effect names
            formatted_effect = effect.replace('C(', '').replace(')', '')
            significant_effects.append((formatted_effect, p_value))
    return significant_effects

# Assuming df_for_trial_level_anova is your DataFrame and it includes a 'SubjectID' column
def perform_modular_within_electrode_anova_whole_brain_analysis(df, save_dir, save_name):
    import json
    results = []
    significant_effects_structure = {}

    for subject_id in df['SubjectID'].unique():
        for electrode in df['Electrode'].unique(): #this is wrong cuz then it only does dlpfc, fix this 3/6
            for time_window in df['TimeWindow'].unique():
                df_filtered = df[(df['SubjectID'] == subject_id) & 
                                 (df['Electrode'] == electrode) & 
                                 (df['TimeWindow'] == time_window)]
                
                if df_filtered.empty:
                    continue
                
                # Dynamically construct the formula based on condition keys present in the DataFrame
                condition_keys = [key for key in output_names_conditions[next(iter(output_names_conditions))].keys()]
                formula_terms = ' + '.join([f'C({key})' for key in condition_keys])
                interaction_terms = ' * '.join([f'C({key})' for key in condition_keys])
                formula = f'Activity ~ {formula_terms} + {interaction_terms}'

                # Perform the ANOVA
                model = ols(formula, data=df_filtered).fit()
                anova_results = anova_lm(model, typ=2)
                
                # Append the results
                results.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode,
                    'TimeWindow': time_window,
                    'ANOVA_Results': anova_results
                })
    
    # Add the suffix '_onlySigElectrodes' to the base filename
    allElectrodesFilename = f"{save_name}_allElectrodes_wholeBrainAnalysis.txt"
    onlySigElectrodesFilename = f"{save_name}_onlySigElectrodes_wholeBrainAnalysis.txt"
    significantEffectsStructureFilename = f"{save_name}_significantEffectsStructure_wholeBrainAnalysis.txt"

    # Define the full path for the results file
    results_file_path = os.path.join(save_dir, allElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(results_file_path, 'w') as file:
        file.write(results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"results saved to: {results_file_path}")

    # Now process the significant results, including the subject ID in the output
    significant_results = []

    for result in results:
        anova_table = result['ANOVA_Results']
        subject_id = result['SubjectID']
        electrode = result['Electrode']
        time_window = result['TimeWindow']
        
        significant_effects = anova_table[anova_table['PR(>F)'] < 0.05]
        
        if not significant_effects.empty:
            print(f"Significant effects found for Subject: {subject_id}, Electrode: {electrode}, Time Window: {time_window}")
            print(significant_effects)
            print("\n")
            
            significant_results.append({
                'SubjectID': subject_id,
                'Electrode': electrode,
                'TimeWindow': time_window,
                'SignificantEffects': significant_effects
            })
        

        # Extract significant effects for the current result. Basically just get the p-value. 3/19.
        sig_effects_just_p_values = extract_significant_effects(anova_table)
        
        if sig_effects_just_p_values:
            # Ensure subject_id and electrode keys exist
            if subject_id not in significant_effects_structure:
                significant_effects_structure[subject_id] = {}
            if electrode not in significant_effects_structure[subject_id]:
                significant_effects_structure[subject_id][electrode] = {}
            
            # Assign the significant effects and their p-values to the correct structure
            significant_effects_structure[subject_id][electrode][time_window] = sig_effects_just_p_values    

    # Define the full path for the results file
    significant_results_file_path = os.path.join(save_dir, onlySigElectrodesFilename)

    # Save the ANOVA results to a text file
    with open(significant_results_file_path, 'w') as file:
        file.write(significant_results.__str__())

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_results saved to: {significant_results_file_path}")

    significant_effects_structure_file_path = os.path.join(save_dir, significantEffectsStructureFilename)
    # Save the ANOVA results to a json file (if this works, change the others to json files too)
    with open(significant_effects_structure_file_path, 'w') as file:
        json.dump(significant_effects_structure, file, indent=4)

    # Optionally, print the path to the saved file and/or return it
    print(f"significant_effects_structure saved to: {significant_effects_structure_file_path}")

    return results, significant_results, significant_effects_structure


In [ ]:
# # For loading json 3/19. Incorporate into plotting code later.
# with open(significant_effects_structure_file_path, 'r') as file:
#     loaded_structure = json.load(file)

congruency as function of congruency proportion  
maybe make the save_name based on the conditions..?

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_whole_brain_analysis(df_for_trial_level_anova, save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

switch type as function of switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_whole_brain_analysis(df_for_trial_level_anova, save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_s25_fixationCrossBase_1sec_mirror' is not in output_names.")

congruency as function of switch type

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    results, significant_results, significant_effects_structure = perform_modular_within_electrode_anova_whole_brain_analysis(df_for_trial_level_anova, save_dir, 'congruency_switchType_ANOVAwithinElectrodes')
else:
    print("The required output name 'Stimulus_c25_fixationCrossBase_1sec_mirror' is not in output_names.")

In [ ]:
significant_effects_structure

In [ ]:
results

In [ ]:
significant_results 

### plot and QC stats

plot time perm cluster stats (don't run this immediately below cell if didn't do time perm cluster)

In [ ]:
# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(time_perm_cluster_results['dlpfc'])
# plt.xlabel('Timepoints')
# plt.ylabel('Significance (0 or 1)')
# plt.title('Permutation Test Significance Over Time')
# plt.show()

### plot interaction effects (only do this when load in all four of them)

https://matplotlib.org/stable/gallery/color/named_colors.html

In [19]:
# add the other conditions and give them condition names and colors too
plotting_parameters = {
    'Stimulus_r25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'repeat',
        'color': 'red',
        "line_style": "-"
    },
    'Stimulus_s25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'switch',
        'color': 'green',
        "line_style": "-"
    },
    'Stimulus_c25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'congruent',
        'color': 'blue',
        "line_style": "-"
    },
    'Stimulus_i25and75_fixationCrossBase_1sec_mirror': {
        'condition_name': 'incongruent',
        'color': 'orange',
        "line_style": "-"
    },
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "condition_name": "IR",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "condition_name": "IS",
        "color": "blue",
        "line_style": "--"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "condition_name": "CR",
        "color": "red",
        "line_style": "-"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "condition_name": "CS",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c25_fixationCrossBase_1sec_mirror": {
        "condition_name": "c25",
        "color": "red",
        "line_style": "--"
    },
    "Stimulus_c75_fixationCrossBase_1sec_mirror": {
        "condition_name": "c75",
        "color": "red",
        "line_style": "-"
    },
    "Stimulus_i25_fixationCrossBase_1sec_mirror": {
        "condition_name": "i25",
        "color": "blue",
        "line_style": "--"
    },
    "Stimulus_i75_fixationCrossBase_1sec_mirror": {
        "condition_name": "i75",
        "color": "blue",
        "line_style": "-"
    },
    "Stimulus_s25_fixationCrossBase_1sec_mirror": {
        "condition_name": "s25",
        "color": "green",
        "line_style": "--"
    },
    "Stimulus_s75_fixationCrossBase_1sec_mirror": {
        "condition_name": "s75",
        "color": "green",
        "line_style": "-"
    },
    "Stimulus_r25_fixationCrossBase_1sec_mirror": {
        "condition_name": "r25",
        "color": "pink",
        "line_style": "--"
    },
    "Stimulus_r75_fixationCrossBase_1sec_mirror": {
        "condition_name": "r75",
        "color": "pink",
        "line_style": "-"
    },

}

In [ ]:
LAB_root = None
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

def plot_interact_effects_modular_whole_brain_analysis(save_dir, save_name, output_names, plotting_parameters):
    # Base setup for directories and file paths
    save_path = os.path.join(save_dir, f'avg_{save_name}_interactEffects_zscore_wholeBrainAnalysis.png')

    # Initialize plot
    plt.figure(figsize=(10, 6))

    # Dynamically select the first subject and use it to extract times
    first_subject_id = next(iter(subjects_mne_objects))
    example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
    times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

    overall_averages_for_plotting = {}
    overall_sem_for_plotting = {}
    # Initialize variables to store the global min and max values
    global_min_val = float('inf')  # Set to infinity initially
    global_max_val = float('-inf')  # Set to negative infinity initially
    
    # Generate labels and plot each condition
    for index, output_name in enumerate(output_names):
        # label = output_name.split("_")[1]  # OR extract label from output name instead of plotting parameters dict. Up to you.
        overall_averages_for_plotting[output_name] = overall_averages[output_name]
        overall_sem_for_plotting[output_name] = overall_sems[output_name]

        # Calculate the minimum value for this condition, including SEM
        current_min_val = min(overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name])
        # Calculate the maximum value for this condition, including SEM
        current_max_val = max(overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name])

        # Update the global min and max values if necessary
        global_min_val = min(global_min_val, current_min_val)
        global_max_val = max(global_max_val, current_max_val)

        # Optionally, add a small margin to the range
        margin = (global_max_val - global_min_val) * 0.05  # 5% of the range as margin
        global_min_val -= margin
        global_max_val += margin

        label = plotting_parameters[output_name]['condition_name'] # extract label from plotting parameters dict
        color = plotting_parameters[output_name]['color']
        line_style = plotting_parameters[output_name]['line_style']

        plt.plot(times, overall_averages_for_plotting[output_name], linestyle=line_style, color=color, label=f'Average {label}')
        plt.fill_between(times, overall_averages_for_plotting[output_name] - overall_sem_for_plotting[output_name], overall_averages_for_plotting[output_name] + overall_sem_for_plotting[output_name], alpha=0.3, color=color)

    plt.xlabel('Time (s)')
    plt.ylabel('Z-score')
    plt.title(f'Average Signal with Standard Error for {save_name}')
    plt.legend()
    # Adjust the y-axis limits
    plt.ylim([global_min_val, global_max_val])
    plt.savefig(save_path)
    plt.show()

this is just for congruent vs congruency proportion

In [ ]:
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_whole_brain_analysis(save_dir, 'congruency_congruencyProportion', output_names, plotting_parameters)

this just for switch vs switch proportion

In [ ]:
if 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_whole_brain_analysis(save_dir, 'switchType_switchProportion', output_names, plotting_parameters)

this just for congruency vs switch type

In [ ]:
if 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    plot_interact_effects_modular_whole_brain_analysis(save_dir, 'congruency_switchType', output_names, plotting_parameters)

### plot individual electrodes for interaction effects
i think this will just work regardless of the output names 3/5

In [20]:
LAB_root = None
channels = None
full_trial_base = False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Dynamically select the first subject and use it to extract times
first_subject_id = next(iter(subjects_mne_objects))
example_output_name = next(iter(subjects_mne_objects[first_subject_id]))
times = subjects_mne_objects[first_subject_id][example_output_name]['HG_ev1_evoke_rescaled'].times

# # Use the times from your evoked data (assuming these are representative for all subjects)
# times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

def plot_significance(ax, times, sig_effects, y_offset=0.1):
    """
    Plot significance bars for the effects on top of the existing axes, adjusted for time windows.

    Parameters:
    - ax: The matplotlib Axes object to plot on.
    - times: Array of time points for the x-axis.
    - sig_effects: Dictionary with time windows as keys and lists of tuples (effect, p-value) as values.
    - y_offset: The vertical offset between different time window significance bars.
    """
    y_pos_base = ax.get_ylim()[1]  # Get the top y-axis limit to place significance bars
    
    time_windows = {
        'FirstHalfSecond': (0, 0.5),
        'SecondHalfSecond': (0.5, 1),
        'FullSecond': (0, 1)
    }

    # Sort time windows to ensure 'FullSecond' bars are plotted last (on top)
    for time_window, effects in sorted(sig_effects.items(), key=lambda x: x[0] == 'FullSecond'):
        y_pos = y_pos_base + y_offset * list(time_windows).index(time_window)  # Adjust y_pos based on time window
        for effect, p_value in effects:
            start_time, end_time = time_windows[time_window]
            # untested new colors 3/20
            # Determine the color based on the effect name
            if 'congruency' in effect:
                color = 'red'
            elif 'congruencyProportion' in effect:
                color = 'blue'
            elif 'switchType' in effect:
                color = 'green'
            elif 'switchProportion' in effect:
                color = 'yellow'
            
            else:
                color = 'black'  # Default color

            # Assign colors for interaction effects
            if 'congruency:congruencyProportion' in effect:
                color = 'purple'
            elif 'switchType:switchProportion' in effect:
                color = 'yellowgreen'
            elif 'congruency:switchType' in effect:
                color = 'brown'

            num_asterisks = '*' * (1 if p_value < 0.05 else 2 if p_value < 0.01 else 3)
            # actually plot
            ax.plot([start_time, end_time], [y_pos, y_pos], color=color, lw=4)
            ax.text((start_time + end_time) / 2, y_pos, num_asterisks, ha='center', va='bottom', color=color)
        

# once these two function are fully working, can move them to misc_functions and then use them in roi analysis too. And can rename to plot_electrodes_grid instead of plot_electrodes_grid_whole_brain_analysis.
def plot_electrodes_grid_whole_brain_analysis(electrodes_data, significant_effects_structure, grid_num, output_names, times, save_dir, save_name, plotting_parameters):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        for output_name in output_names:
            color = plotting_parameters[output_name]['color']
            line_style = plotting_parameters[output_name]['line_style']
            ax.plot(times, data[output_name], label=f'{output_name}', color=color, linestyle=line_style)

            # i think this is wrong..? It's getting the standard deviation of the trial averaged data, but I want the stdev of the trials themselves for each timepoint.
            # So maybe this is a constant stdev across time? Idk. Try using the trial_std 3/17. 
            ax.fill_between(times, 
                            data[output_name] - np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])),
                            data[output_name] + np.std(data[output_name], ddof=1) / np.sqrt(len(data[output_name])), alpha=0.3)

        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

        # Retrieve significant effects for the current subject and electrode
        sig_effects = significant_effects_structure.get(sub, {}).get(electrode, {})
        if sig_effects:
            # Adjust y_offset based on plotting needs. This used to not be assigned to a variable. 3/20.
            plot_significance(ax, times, sig_effects, y_offset=0.1)

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'wholeBrainAnalysis_{save_name}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()

# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
if 'Stimulus_c25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_congruencyProportion' # i think this will be congruency x con prop if i load in c25?
elif 'Stimulus_s25_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'switchType_switchProportion' # i think if there's no c25, but there is s25, then i am doing switch x switch prop? 3/17
elif 'Stimulus_cr_fixationCrossBase_1sec_mirror' in output_names:
    save_name = 'congruency_switchType'
    
significant_effects_structure_file_path = os.path.join(save_dir, f'{save_name}_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(significant_effects_structure_file_path, 'r') as file:
    significant_effects_structure = json.load(file)
    
for sub in subjects:
    for electrode in sig_chans_per_subject[sub]:
        
        electrode_data = {}
        for output_name in output_names:
            electrode_data[output_name] = concatenated_trialAvg_data[output_name][electrode_counter]

        electrodes_data.append((electrode_data, sub, electrode))
        electrode_counter += 1
        if len(electrodes_data) == grid_size:

            plot_electrodes_grid_whole_brain_analysis(electrodes_data, significant_effects_structure, grid_num, output_names, times, save_dir, save_name, plotting_parameters)
            electrodes_data = []  # Reset for the next grid
            grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid_whole_brain_analysis(electrodes_data, significant_effects_structure, grid_num, output_names, times, save_dir, save_name, plotting_parameters)

IndexError: index 839 is out of bounds for axis 0 with size 839

In [ ]:
significant_effects_structure

make a legend to put on the powerpoint slide for significance bars

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def create_significance_legend(effect_color_mapping, save_dir, filename="significance_legend.png"):
    """
    Create and save a separate legend for significance colors.

    Parameters:
    - effect_color_mapping: Dictionary mapping effect names to colors.
    - save_dir: Directory where the legend PNG will be saved.
    - filename: Name of the PNG file for the legend.
    """
    # Create a figure and a dummy subplot (not displayed) to host the legend
    fig, ax = plt.subplots()
    ax.axis('off')  # Turn off the axis

    # Create legend handles
    legend_elements = [
        Line2D([0], [0], color=color, lw=4, label=effect)
        for effect, color in effect_color_mapping.items()
    ]
    
    # Create the legend
    legend = ax.legend(handles=legend_elements, loc='center', frameon=False)

    # Save the legend as a separate figure
    fig.savefig(os.path.join(save_dir, filename), bbox_inches='tight')
    plt.close(fig)  # Close the figure window

# Example usage
effect_color_mapping = {
    'congruency': 'red',
    'congruencyProportion': 'blue',
    'switchType': 'green',
    'switchProportion': 'yellow',
    'congruency:congruencyProportion': 'purple',
    'switchType:switchProportion': 'yellowgreen',
    'congruency:switchType': 'brown'
}

# Assuming you have a save directory defined as save_dir
create_significance_legend(effect_color_mapping, save_dir)


there are 839 electrodes in concatenated_trialAvg_data. BUT there are 824 electrodes in overall electrode mapping, and also 842 electrodes in sig chans per subject. What is going on here..?

In [ ]:
# Checking the size of each array in concatenated_trialAvg_data
sizes = {key: data.shape for key, data in concatenated_trialAvg_data.items()}

sizes

In [ ]:
total_electrodes = sum(len(electrodes) for electrodes in sig_chans_per_subject.values())
total_electrodes